# StartUp

In [1]:
using JuMP
using Gurobi

# Constants

In [2]:
Days = 16;
Times = 4;

StadiumCountR1 = 4;
StadiumCountR2 = 7;
StadiumCountR3 = 5;

StadiumsR1 = 1:4;
StadiumsR2 = 5:11;
StadiumsR3 = 12:16;

CountryCountR1 = 12;
CountryCountR2 = 20;
CountryCountR3 = 16;

CountriesR1 = 1:12;
CountriesR2 = 13:32;
CountriesR3 = 33:48;

#new coefficient for round robbins that I came up with after a comment that each team plays 3 games and I thought they were 6
# RR stands for Round Robin
RR = 3;

VR1 = [
    [0 120 800 1080];
    [140 0 680 960];
    [800 680 0 380]; 
    [1050 960 380 0]
    ];

VR2 = [
    [0 570 340 930 940 1160 1330]; 
    [570 0 420 610 510 780 1020];
    [340 420 0 940 880 1050 1220];
    [930 610 940 0 240 460 780];
    [940 510 880 240 0 650 800];
    [1160 780 1050 460 650 0 800];
    [1330 1020 1220 780 800 800 0]
    ];

VR3 = [
    [0 550 490 510 1480];
    [550 0 220 310 1250]; 
    [490 220 0 100 1280];
    [510 310 100 0 1210];
    [1480 1250 1280 1210 0]
    ];

LD = [1 3 4 4 4 4 4 4 4 4 4 4 4 6 6 6 6];

# Gurobi Model For Region 1

In [3]:
modelR1 = Model(Gurobi.Optimizer);


Set parameter Username
Set parameter LicenseID to value 2685810
Academic license - for non-commercial use only - expires 2026-07-09


## Variables R1

In [4]:
@variable(modelR1, X[c in CountriesR1, s in StadiumsR1, d in 1:Days, t in 1:Times], Bin);   
@variable(modelR1, Y[i in CountriesR1, j in CountriesR1, d in 1:Days], Bin);  

## Objective R1

In [5]:
@objective(modelR1, Min, sum(VR1[dsi, dsj] * X[c, i, d1, t1] * X[c, j, d2, t2] 
        for dsi in 1:(StadiumCountR1 - 1), dsj in 1:(StadiumCountR1 - 1), 
            i in StadiumsR1, j in StadiumsR1, c in CountriesR1, 
            d1 in 1:Days, d2 in 1:Days, t1 in 1:Times, t2 in 1:Times))

3220 X[1,1,1,1]² + 6440 X[1,1,1,1]*X[1,1,1,2] + 6440 X[1,1,1,1]*X[1,1,1,3] + 6440 X[1,1,1,1]*X[1,1,1,4] + 3220 X[1,1,1,2]² + 6440 X[1,1,1,2]*X[1,1,1,3] + 6440 X[1,1,1,2]*X[1,1,1,4] + 3220 X[1,1,1,3]² + 6440 X[1,1,1,3]*X[1,1,1,4] + 3220 X[1,1,1,4]² + 6440 X[1,1,1,1]*X[1,1,2,1] + 6440 X[1,1,1,1]*X[1,1,2,2] + 6440 X[1,1,1,1]*X[1,1,2,3] + 6440 X[1,1,1,1]*X[1,1,2,4] + 6440 X[1,1,1,2]*X[1,1,2,1] + 6440 X[1,1,1,2]*X[1,1,2,2] + 6440 X[1,1,1,2]*X[1,1,2,3] + 6440 X[1,1,1,2]*X[1,1,2,4] + 6440 X[1,1,1,3]*X[1,1,2,1] + 6440 X[1,1,1,3]*X[1,1,2,2] + 6440 X[1,1,1,3]*X[1,1,2,3] + 6440 X[1,1,1,3]*X[1,1,2,4] + 6440 X[1,1,1,4]*X[1,1,2,1] + 6440 X[1,1,1,4]*X[1,1,2,2] + 6440 X[1,1,1,4]*X[1,1,2,3] + 6440 X[1,1,1,4]*X[1,1,2,4] + 6440 X[1,1,1,1]*X[1,1,3,1] + 6440 X[1,1,1,1]*X[1,1,3,2] + 6440 X[1,1,1,1]*X[1,1,3,3] + 6440 X[1,1,1,1]*X[1,1,3,4] + [[...394692 terms omitted...]] + 6440 X[12,4,15,2]*X[12,4,15,4] + 3220 X[12,4,15,3]² + 6440 X[12,4,15,3]*X[12,4,15,4] + 3220 X[12,4,15,4]² + 6440 X[12,4,15,1]*X[12,4,16,1

## Constraints R1

### Model Breakers R1

In [6]:
# Round Robin 3 Matches for each Country
@constraint(modelR1, [c in CountriesR1], sum(X[c, s, d, t] 
        for s in StadiumsR1, d in 1:Days, t in 1:Times) == RR)

# Match Ensurinng Constraint
@constraint(modelR1, [i in CountriesR1, j in CountriesR1, d in 1:Days], 
    sum(X[i, s, d, t] * X[j, s, d, t] 
        for s in StadiumsR1, t in 1:Times) <= Y[i, j, d])

# Team doesn't play itself, deprecated constraint but I still wanted to keep to keep it around in my notes 
# @constraint(model, [C in Countries], sum(X[C, S, D, T] for S in Stadiums, D in Days, T in Times) == 0)

3-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},3,...} with index sets:
    Dimension 1, 1:12
    Dimension 2, 1:12
    Dimension 3, Base.OneTo(16)
And data, a 12×12×16 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 3}:
[:, :, 1] =
 X[1,1,1,1]² + X[1,2,1,1]² + X[1,3,1,1]² + X[1,4,1,1]² + X[1,1,1,2]² + X[1,2,1,2]² + X[1,3,1,2]² + X[1,4,1,2]² + X[1,1,1,3]² + X[1,2,1,3]² + X[1,3,1,3]² + X[1,4,1,3]² + X[1,1,1,4]² + X[1,2,1,4]² + X[1,3,1,4]² + X[1,4,1,4]² - Y[1,1,1] ≤ 0                                                                                                                                                                                   …  X[1,1,1,1]*X[12,1,1,1] + X[1,2,1,1]*X[12,2,1,1] + X[1,3,1,1]*X[12,3,1,1] + X[1,4,1,1]*X[12,4,1,1] 

### Special Constraints Outlined by Rubric R1

In [7]:

# US plays June 12 @ SoFi Stadium 8 PM
@constraint(modelR1, X[1, 4, 2, 3] == 1)

# Daily match limit
@constraint(modelR1, [d in 3:Days], 
    sum(Y[i, j, d] 
        for i in CountriesR1, j in CountriesR1) <= LD[d])

# Each stadium hosts at least 1 match
@constraint(modelR1, [s in StadiumsR1], 
        sum(X[c, s, d, t] 
        for c in CountriesR1, d in 1:Days, t in 1:Times) >= 1)

# Balanced Match Distribution
@constraint(modelR1, [s in StadiumsR1[1]:StadiumsR1[StadiumCountR1-1]], 
    sum(X[c, s, d, t] for c in CountriesR1, d in 1:Days, t in 1:Times) 
    - sum(X[c, s+1, d, t] for c in CountriesR1, d in 1:Days, t in 1:Times) <= 2)

# Team plays one match then rest two days at least
@constraint(modelR1, [d in 1:(Days - 2)],
    sum(X[c, s, d, t] for c in CountriesR1, s in StadiumsR1, t in 1:Times) 
    + sum(X[c, s, d+1, t] for c in CountriesR1, s in StadiumsR1, t in 1:Times) 
    + sum(X[c, s, d+2, t] for c in CountriesR1, s in StadiumsR1, t in 1:Times) == 1)

# Team has at most one match per day
@constraint(modelR1, [d in 1:Days, c in CountriesR1], 
        sum(X[c, s, d, t] 
        for s in StadiumsR1, t in 1:Times) <= 1)

# No overlapping Matches at same stadium at same time
@constraint(modelR1, [s in StadiumsR1, d in 1:Days, t in 1:Times], 
        sum(X[c, s, d, t] 
        for c in CountriesR1) <= 2)

# Host Nations only play in the 8 PM Slot by making all other times 0

#USA
@constraint(modelR1, sum(X[1, s, d, 1] for s in StadiumsR1, d in 1:Days) 
    + sum(X[1, s, d, 2] for s in StadiumsR1, d in 1:Days) == 0)


# Symmetry constraint suggestion
@constraint(modelR1, [i in CountriesR1, j in CountriesR1, d in 1:Days], Y[i, j, d] + Y[j, i, d] <= 1)

# 3 Unique oponents, or RR unique oponents with that new coefficient that is more like a variable at this point
@constraint(modelR1, [i in CountriesR1, j in CountriesR1], sum(Y[i, j, d] + Y[j, i, d] 
        for d in 1:Days) == 1) #Symmetry

@constraint(modelR1, [i in CountriesR1, j in CountriesR1], sum(Y[i, j, d] 
        for d in 1:Days) <= 1) # Matches don't repeat on a different day

@constraint(modelR1, [i in CountriesR1], sum(Y[i, i, d] 
        for d in 1:Days) == 0) # Team doesn't play against itself, ever  

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:12
And data, a 12-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 Y[1,1,1] + Y[1,1,2] + Y[1,1,3] + Y[1,1,4] + Y[1,1,5] + Y[1,1,6] + Y[1,1,7] + Y[1,1,8] + Y[1,1,9] + Y[1,1,10] + Y[1,1,11] + Y[1,1,12] + Y[1,1,13] + Y[1,1,14] + Y[1,1,15] + Y[1,1,16] = 0
 Y[2,2,1] + Y[2,2,2] + Y[2,2,3] + Y[2,2,4] + Y[2,2,5] + Y[2,2,6] + Y[2,2,7] + Y[2,2,8] + Y[2,2,9] + Y[2,2,10] + Y[2,2,11] + Y[2,2,12] + Y[2,2,13] + Y[2,2,14] + Y[2,2,15] + Y[2,2,16] = 0
 Y[3,3,1] + Y[3,3,2] + Y[3,3,3] + Y[3,3,4] + Y[3,3,5] + Y[3,3,6] + Y[3,3,7] + Y[3,3,8] + Y[3,3,9] + Y[3,3,10] + Y[3,3,11] + Y[3,3,12] + Y[3,3,13] + Y[3,3,14] + Y[3,3,15] + Y[3,3,16] = 0
 Y[4,4,1] + Y[4,4,2] + Y[4,4,3] +

In [8]:
optimize!(modelR1)

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 3101 rows, 5376 columns and 38433 nonzeros
Model fingerprint: 0x341efc46
Model has 394752 quadratic objective terms
Model has 2304 quadratic constraints
Variable types: 0 continuous, 5376 integer (5376 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e+03, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+00]
Presolve added 192 rows and 0 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective 

# Gurobi Model For Region 2

In [9]:
modelR2 = Model(Gurobi.Optimizer);


Set parameter Username
Set parameter LicenseID to value 2685810
Academic license - for non-commercial use only - expires 2026-07-09


## Variables R2

In [10]:
@variable(modelR2, X[c in CountriesR2, s in StadiumsR2, d in 1:Days, t in 1:Times], Bin);   
@variable(modelR2, Y[i in CountriesR2, j in CountriesR2, d in 1:Days], Bin);  

## Objective R2

In [11]:
@objective(modelR2, Min, sum(VR2[dsi, dsj] * X[c, i, d1, t1] * X[c, j, d2, t2] 
        for dsi in 1:(StadiumCountR2 - 1), dsj in 1:(StadiumCountR2 - 1), 
            i in StadiumsR2, j in StadiumsR2, c in CountriesR2, 
            d1 in 1:Days, d2 in 1:Days, t1 in 1:Times, t2 in 1:Times))


20960 X[13,5,1,1]² + 41920 X[13,5,1,1]*X[13,5,1,2] + 41920 X[13,5,1,1]*X[13,5,1,3] + 41920 X[13,5,1,1]*X[13,5,1,4] + 20960 X[13,5,1,2]² + 41920 X[13,5,1,2]*X[13,5,1,3] + 41920 X[13,5,1,2]*X[13,5,1,4] + 20960 X[13,5,1,3]² + 41920 X[13,5,1,3]*X[13,5,1,4] + 20960 X[13,5,1,4]² + 41920 X[13,5,1,1]*X[13,5,2,1] + 41920 X[13,5,1,1]*X[13,5,2,2] + 41920 X[13,5,1,1]*X[13,5,2,3] + 41920 X[13,5,1,1]*X[13,5,2,4] + 41920 X[13,5,1,2]*X[13,5,2,1] + 41920 X[13,5,1,2]*X[13,5,2,2] + 41920 X[13,5,1,2]*X[13,5,2,3] + 41920 X[13,5,1,2]*X[13,5,2,4] + 41920 X[13,5,1,3]*X[13,5,2,1] + 41920 X[13,5,1,3]*X[13,5,2,2] + 41920 X[13,5,1,3]*X[13,5,2,3] + 41920 X[13,5,1,3]*X[13,5,2,4] + 41920 X[13,5,1,4]*X[13,5,2,1] + 41920 X[13,5,1,4]*X[13,5,2,2] + 41920 X[13,5,1,4]*X[13,5,2,3] + 41920 X[13,5,1,4]*X[13,5,2,4] + 41920 X[13,5,1,1]*X[13,5,3,1] + 41920 X[13,5,1,1]*X[13,5,3,2] + 41920 X[13,5,1,1]*X[13,5,3,3] + 41920 X[13,5,1,1]*X[13,5,3,4] + [[...2011460 terms omitted...]] + 41920 X[32,11,15,2]*X[32,11,15,4] + 20960 X[32,11,

## Constraints R2

### Model Breakers R2

In [12]:
# Round Robin 3 Matches for each Country
@constraint(modelR2, [c in CountriesR2], sum(X[c, s, d, t] 
        for s in StadiumsR2, d in 1:Days, t in 1:Times) == RR)

# Match Ensurinng Constraint
@constraint(modelR2, [i in CountriesR2, j in CountriesR2, d in 1:Days], 
    sum(X[i, s, d, t] * X[j, s, d, t] 
        for s in StadiumsR2, t in 1:Times) <= Y[i, j, d])


3-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},3,...} with index sets:
    Dimension 1, 13:32
    Dimension 2, 13:32
    Dimension 3, Base.OneTo(16)
And data, a 20×20×16 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 3}:
[:, :, 1] =
 X[13,5,1,1]² + X[13,6,1,1]² + X[13,7,1,1]² + X[13,8,1,1]² + X[13,9,1,1]² + X[13,10,1,1]² + X[13,11,1,1]² + X[13,5,1,2]² + X[13,6,1,2]² + X[13,7,1,2]² + X[13,8,1,2]² + X[13,9,1,2]² + X[13,10,1,2]² + X[13,11,1,2]² + X[13,5,1,3]² + X[13,6,1,3]² + X[13,7,1,3]² + X[13,8,1,3]² + X[13,9,1,3]² + X[13,10,1,3]² + X[13,11,1,3]² + X[13,5,1,4]² + X[13,6,1,4]² + X[13,7,1,4]² + X[13,8,1,4]² + X[13,9,1,4]² + X[13,10,1,4]² + X[13,11,1,4]² - Y[13,13,1] ≤ 0                                                                        

### Special Constraints Outlined by Rubric R2

In [13]:
# MX plays June 11 @ Estadio Azteca 8 PM
@constraint(modelR2, X[13, 5, 1, 3] == 1)

# Daily match limit
@constraint(modelR2, [d in 3:Days], 
    sum(Y[i, j, d] 
        for i in CountriesR2, j in CountriesR2) <= LD[d])

# Each stadium hosts at least 1 match
@constraint(modelR2, [s in StadiumsR2], 
        sum(X[c, s, d, t] 
        for c in CountriesR2, d in 1:Days, t in 1:Times) >= 1)

# Balanced Match Distribution
@constraint(modelR2, [s in StadiumsR2[1]:StadiumsR2[StadiumCountR2-1]], 
    sum(X[c, s, d, t] for c in CountriesR2, d in 1:Days, t in 1:Times) 
    - sum(X[c, s+1, d, t] for c in CountriesR2, d in 1:Days, t in 1:Times) <= 2)

# Team plays one match then rest two days at least
@constraint(modelR2, [d in 1:(Days - 2)],
    sum(X[c, s, d, t] for c in CountriesR2, s in StadiumsR2, t in 1:Times) 
    + sum(X[c, s, d+1, t] for c in CountriesR2, s in StadiumsR2, t in 1:Times) 
    + sum(X[c, s, d+2, t] for c in CountriesR2, s in StadiumsR2, t in 1:Times) == 1)

# Team has at most one match per day
@constraint(modelR2, [d in 1:Days, c in CountriesR2], 
        sum(X[c, s, d, t] 
        for s in StadiumsR2, t in 1:Times) <= 1)

# No overlapping Matches at same stadium at same time
@constraint(modelR2, [s in StadiumsR2, d in 1:Days, t in 1:Times], 
        sum(X[c, s, d, t] 
        for c in CountriesR2) <= 2)

# Host Nations only play in the 8 PM Slot by making all other times 0
# MX
@constraint(modelR2, sum(X[13, s, d, 1] for s in StadiumsR2, d in 1:Days) 
    + sum(X[13, s, d, 2] for s in StadiumsR2, d in 1:Days) == 0)

# Symmetry constraint suggestion
@constraint(modelR2, [i in CountriesR2, j in CountriesR2, d in 1:Days], Y[i, j, d] + Y[j, i, d] <= 1)

# 3 Unique oponents, or RR unique oponents with that new coefficient that is more like a variable at this point
@constraint(modelR2, [i in CountriesR2, j in CountriesR2], sum(Y[i, j, d] + Y[j, i, d] 
        for d in 1:Days) == 1) #Symmetry

@constraint(modelR2, [i in CountriesR2, j in CountriesR2], sum(Y[i, j, d] 
        for d in 1:Days) <= 1) # Matches don't repeat on a different day

@constraint(modelR2, [i in CountriesR2], sum(Y[i, i, d] 
        for d in 1:Days) == 0) # Team doesn't play against itself, ever  


1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 13:32
And data, a 20-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 Y[13,13,1] + Y[13,13,2] + Y[13,13,3] + Y[13,13,4] + Y[13,13,5] + Y[13,13,6] + Y[13,13,7] + Y[13,13,8] + Y[13,13,9] + Y[13,13,10] + Y[13,13,11] + Y[13,13,12] + Y[13,13,13] + Y[13,13,14] + Y[13,13,15] + Y[13,13,16] = 0
 Y[14,14,1] + Y[14,14,2] + Y[14,14,3] + Y[14,14,4] + Y[14,14,5] + Y[14,14,6] + Y[14,14,7] + Y[14,14,8] + Y[14,14,9] + Y[14,14,10] + Y[14,14,11] + Y[14,14,12] + Y[14,14,13] + Y[14,14,14] + Y[14,14,15] + Y[14,14,16] = 0
 Y[15,15,1] + Y[15,15,2] + Y[15,15,3] + Y[15,15,4] + Y[15,15,5] + Y[15,15,6] + Y[15,15,7] + Y[15,15,8] + Y[15,15,9] + Y[15,15,10] + Y[15,15,11] + Y[15,15,

In [14]:
optimize!(modelR2)

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 8051 rows, 15360 columns and 112225 nonzeros
Model fingerprint: 0x2395f330
Model has 2011520 quadratic objective terms
Model has 6400 quadratic constraints
Variable types: 0 continuous, 15360 integer (15360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+04, 8e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+00]
Presolve added 320 rows and 0 columns
Presolve time: 0.06s

Explored 0 nodes (0 simplex iterations) in 0.08 seconds (0.06 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible or unbounded
Best objec

# Gurobi Model For Region 3

In [15]:
modelR3 = Model(Gurobi.Optimizer);


Set parameter Username
Set parameter LicenseID to value 2685810
Academic license - for non-commercial use only - expires 2026-07-09


## Variables R3

In [16]:
@variable(modelR3, X[c in CountriesR3, s in StadiumsR3, d in 1:Days, t in 1:Times], Bin);   
@variable(modelR3, Y[i in CountriesR3, j in CountriesR3, d in 1:Days], Bin);  


## Objective R3

In [17]:
@objective(modelR3, Min, sum(VR3[dsi, dsj] * X[c, i, d1, t1] * X[c, j, d2, t2] 
        for dsi in 1:(StadiumCountR3 - 1), dsj in 1:(StadiumCountR3 - 1), 
            i in StadiumsR3, j in StadiumsR3, c in CountriesR3, 
            d1 in 1:Days, d2 in 1:Days, t1 in 1:Times, t2 in 1:Times))


4360 X[33,12,1,1]² + 8720 X[33,12,1,1]*X[33,12,1,2] + 8720 X[33,12,1,1]*X[33,12,1,3] + 8720 X[33,12,1,1]*X[33,12,1,4] + 4360 X[33,12,1,2]² + 8720 X[33,12,1,2]*X[33,12,1,3] + 8720 X[33,12,1,2]*X[33,12,1,4] + 4360 X[33,12,1,3]² + 8720 X[33,12,1,3]*X[33,12,1,4] + 4360 X[33,12,1,4]² + 8720 X[33,12,1,1]*X[33,12,2,1] + 8720 X[33,12,1,1]*X[33,12,2,2] + 8720 X[33,12,1,1]*X[33,12,2,3] + 8720 X[33,12,1,1]*X[33,12,2,4] + 8720 X[33,12,1,2]*X[33,12,2,1] + 8720 X[33,12,1,2]*X[33,12,2,2] + 8720 X[33,12,1,2]*X[33,12,2,3] + 8720 X[33,12,1,2]*X[33,12,2,4] + 8720 X[33,12,1,3]*X[33,12,2,1] + 8720 X[33,12,1,3]*X[33,12,2,2] + 8720 X[33,12,1,3]*X[33,12,2,3] + 8720 X[33,12,1,3]*X[33,12,2,4] + 8720 X[33,12,1,4]*X[33,12,2,1] + 8720 X[33,12,1,4]*X[33,12,2,2] + 8720 X[33,12,1,4]*X[33,12,2,3] + 8720 X[33,12,1,4]*X[33,12,2,4] + 8720 X[33,12,1,1]*X[33,12,3,1] + 8720 X[33,12,1,1]*X[33,12,3,2] + 8720 X[33,12,1,1]*X[33,12,3,3] + 8720 X[33,12,1,1]*X[33,12,3,4] + [[...821700 terms omitted...]] + 8720 X[48,16,15,2]*X[48,1

## Constraints R3

### Model Breakers R3

In [18]:
# Round Robin 3 Matches for each Country
@constraint(modelR3, [c in CountriesR3], sum(X[c, s, d, t] 
        for s in StadiumsR3, d in 1:Days, t in 1:Times) == RR)

# Match Ensurinng Constraint
@constraint(modelR3, [i in CountriesR3, j in CountriesR3, d in 1:Days], 
    sum(X[i, s, d, t] * X[j, s, d, t] 
        for s in StadiumsR3, t in 1:Times) <= Y[i, j, d])



3-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},3,...} with index sets:
    Dimension 1, 33:48
    Dimension 2, 33:48
    Dimension 3, Base.OneTo(16)
And data, a 16×16×16 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 3}:
[:, :, 1] =
 X[33,12,1,1]² + X[33,13,1,1]² + X[33,14,1,1]² + X[33,15,1,1]² + X[33,16,1,1]² + X[33,12,1,2]² + X[33,13,1,2]² + X[33,14,1,2]² + X[33,15,1,2]² + X[33,16,1,2]² + X[33,12,1,3]² + X[33,13,1,3]² + X[33,14,1,3]² + X[33,15,1,3]² + X[33,16,1,3]² + X[33,12,1,4]² + X[33,13,1,4]² + X[33,14,1,4]² + X[33,15,1,4]² + X[33,16,1,4]² - Y[33,33,1] ≤ 0                                                                                                                                                                                    

### Special Constraints Outlined by Rubric R3

In [19]:
# CAN plays June 12 @ BMO Field 8 PM
@constraint(modelR3, X[33, 12, 2, 3] == 1)

# Daily match limit
@constraint(modelR3, [d in 3:Days], 
    sum(Y[i, j, d] 
        for i in CountriesR3, j in CountriesR3) <= LD[d])

# Each stadium hosts at least 1 match
@constraint(modelR3, [s in StadiumsR3], 
        sum(X[c, s, d, t] 
        for c in CountriesR3, d in 1:Days, t in 1:Times) >= 1)

# Balanced Match Distribution
@constraint(modelR3, [s in StadiumsR3[1]:StadiumsR3[StadiumCountR3-1]], 
    sum(X[c, s, d, t] for c in CountriesR3, d in 1:Days, t in 1:Times) 
    - sum(X[c, s+1, d, t] for c in CountriesR3, d in 1:Days, t in 1:Times) <= 2)

# Team plays one match then rest two days at least
@constraint(modelR3, [d in 1:(Days - 2)],
    sum(X[c, s, d, t] for c in CountriesR3, s in StadiumsR3, t in 1:Times) 
    + sum(X[c, s, d+1, t] for c in CountriesR3, s in StadiumsR3, t in 1:Times) 
    + sum(X[c, s, d+2, t] for c in CountriesR3, s in StadiumsR3, t in 1:Times) == 1)

# Team has at most one match per day
@constraint(modelR3, [d in 1:Days, c in CountriesR3], 
        sum(X[c, s, d, t] 
        for s in StadiumsR3, t in 1:Times) <= 1)

# No overlapping Matches at same stadium at same time
@constraint(modelR3, [s in StadiumsR3, d in 1:Days, t in 1:Times], 
        sum(X[c, s, d, t] 
        for c in CountriesR3) <= 2)

# Host Nations only play in the 8 PM Slot by making all other times 0
#CAN 
@constraint(modelR3, sum(X[33, s, d, 1] for s in StadiumsR3, d in 1:Days) 
    + sum(X[33, s, d, 2] for s in StadiumsR3, d in 1:Days) == 0)

# Symmetry constraint suggestion
@constraint(modelR3, [i in CountriesR3, j in CountriesR3, d in 1:Days], Y[i, j, d] + Y[j, i, d] <= 1)

# 3 Unique oponents, or RR unique oponents with that new coefficient that is more like a variable at this point
@constraint(modelR3, [i in CountriesR3, j in CountriesR3], sum(Y[i, j, d] + Y[j, i, d] 
        for d in 1:Days) == 1) #Symmetry

@constraint(modelR3, [i in CountriesR3, j in CountriesR3], sum(Y[i, j, d] 
        for d in 1:Days) <= 1) # Matches don't repeat on a different day

@constraint(modelR3, [i in CountriesR3], sum(Y[i, i, d] 
        for d in 1:Days) == 0) # Team doesn't play against itself, ever  


1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 33:48
And data, a 16-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 Y[33,33,1] + Y[33,33,2] + Y[33,33,3] + Y[33,33,4] + Y[33,33,5] + Y[33,33,6] + Y[33,33,7] + Y[33,33,8] + Y[33,33,9] + Y[33,33,10] + Y[33,33,11] + Y[33,33,12] + Y[33,33,13] + Y[33,33,14] + Y[33,33,15] + Y[33,33,16] = 0
 Y[34,34,1] + Y[34,34,2] + Y[34,34,3] + Y[34,34,4] + Y[34,34,5] + Y[34,34,6] + Y[34,34,7] + Y[34,34,8] + Y[34,34,9] + Y[34,34,10] + Y[34,34,11] + Y[34,34,12] + Y[34,34,13] + Y[34,34,14] + Y[34,34,15] + Y[34,34,16] = 0
 Y[35,35,1] + Y[35,35,2] + Y[35,35,3] + Y[35,35,4] + Y[35,35,5] + Y[35,35,6] + Y[35,35,7] + Y[35,35,8] + Y[35,35,9] + Y[35,35,10] + Y[35,35,11] + Y[35,35,

In [20]:
optimize!(modelR3)

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5255 rows, 9216 columns and 66081 nonzeros
Model fingerprint: 0x9e71a53d
Model has 821760 quadratic objective terms
Model has 4096 quadratic constraints
Variable types: 0 continuous, 9216 integer (9216 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [9e+03, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+00]
Presolve added 256 rows and 0 columns
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.03 work units)
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective 